In [1]:
import os
import glob
import numpy as np
import pandas as pd
from functions import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
os.chdir("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/files/Datasets")

Data Taken from http://www.football-data.co.uk/englandm.php

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
#combine all files in the list
master_sheet = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
master_sheet.to_csv( "master_sheet.csv", index=False, encoding='utf-8-sig')

Reduce table to using columns needed for the model

Columns Required:

Date, Time HomeTeam, AwayTeam, FullTimeHG, FullTimeAG, FullTimeRresults, HalfTimeHG, HalfTimeAG, HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

Columns To Consider:

Attendance, Referee

In [4]:
subset_master = master_sheet[["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HR", "AR"]]
subset_master = subset_master.fillna(0)
# 0 = Home Wins, 1 = Draw, 2 = Away Wins
subset_master.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)

In [5]:
subset_master.head()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,13.0,8.0,4.0,12.0,6.0,14.0,10.0,0.0,0.0
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,7.0,9.0,1.0,2.0,3.0,10.0,18.0,0.0,0.0
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,15.0,4.0,6.0,6.0,3.0,17.0,12.0,0.0,0.0
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,9.0,1.0,4.0,2.0,5.0,9.0,11.0,0.0,1.0
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,13.0,7.0,7.0,3.0,2.0,9.0,16.0,1.0,0.0


In [6]:
msk = np.random.rand(len(subset_master)) < 0.75

train_df = subset_master[msk]
test_df = subset_master[~msk]

X_train = train_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG"])
y_train = train_df[['FTR']] 
X_test = test_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG", ])
y_test = test_df[['FTR']]

In [7]:
clf = RandomForestClassifier(n_estimators = 5000, max_depth=3, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)

Identifying how often Predicted Results decided the outcome (win, draw, away) against the actual results)

In [8]:
metrics(subset_master, y_pred)

False    58.924417
True     41.075583
Name: Comparison, dtype: float64
0    45.347694
2    30.221401
1    24.430905
Name: FTR, dtype: float64
0    74.319365
2    25.680635
Name: PredResult, dtype: float64


In [9]:
subset_master

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,...,4.0,12.0,6.0,14.0,10.0,0.0,0.0,0.519734,0.237229,0.243037
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,...,1.0,2.0,3.0,10.0,18.0,0.0,0.0,0.558578,0.215595,0.225827
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,...,6.0,6.0,3.0,17.0,12.0,0.0,0.0,0.447041,0.237071,0.315887
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,...,4.0,2.0,5.0,9.0,11.0,0.0,1.0,0.586295,0.212948,0.200758
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,...,7.0,3.0,2.0,9.0,16.0,1.0,0.0,0.482060,0.226718,0.291222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,24/05/15,0,Leicester,QPR,5.0,1.0,0,2.0,0.0,22.0,...,2.0,5.0,6.0,7.0,6.0,0.0,0.0,0.393982,0.288009,0.318009
377,24/05/15,0,Man City,Southampton,2.0,0.0,0,1.0,0.0,15.0,...,4.0,8.0,4.0,13.0,8.0,0.0,0.0,0.231824,0.245533,0.522643
378,24/05/15,0,Newcastle,West Ham,2.0,0.0,0,0.0,0.0,17.0,...,1.0,2.0,3.0,9.0,9.0,0.0,0.0,0.274482,0.253334,0.472184
379,24/05/15,0,Stoke,Liverpool,6.0,1.0,0,5.0,0.0,15.0,...,4.0,3.0,9.0,13.0,4.0,0.0,0.0,0.488295,0.232633,0.279072


In [10]:
subset_master.columns

Index(['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HR', 'AR',
       'HomeWinPerc', 'DrawPerc', 'AwayPerc'],
      dtype='object')

Columns Required for predicting:

HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

In [11]:
#clf.predict(['HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HR', 'AR'])

#clf.predict([8, 4, 6, 2, 6, 5, 3, 6, 0, 0])
